# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Introdução à Inteligência Artificial</font>

## Lógica

Começaremos examinando `Expr`, o tipo de dados para sentenças lógicas e a função de conveniência ` expr` e então vamos cobrir `KB` e` ProbKB`, as classes para Knowledge Bases. Na sequência, vamos construir uma base de conhecimento de uma situação específica no Mundo de Wumpus (mais detalhes no próximo vídeo). Vamos passar pela função `tt_entails` e experimentar um pouco. As funções `pl_resolution` e` pl_fc_entails` virão a seguir.

Mas o primeiro passo é carregar o código:

In [1]:
from utils import *
from logica import *

## Sentenças Lógicas

A classe `Expr` é projetada para representar qualquer tipo de expressão matemática. O tipo mais simples de `Expr` é um símbolo, que pode ser definido com a função` Symbol`:

In [2]:
Symbol('x')

x

Ou podemos definir vários símbolos ao mesmo tempo com a função `symbols`:

In [3]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

Podemos combinar `Expr`s com os operadores regulares de infix e prefix do Python. Aqui está como nós formaríamos a frase lógica "P e não Q":

In [4]:
P & ~Q

(P & ~Q)

Isso funciona porque a classe `Expr` sobrecarrega o operador ` & ` com esta definição:

```python
def __and__(self, other): return Expr('&',  self, other)```
     
E faz sobrecargas semelhantes para os outros operadores. Um `Expr` tem dois campos: ` op` para o operador, que é sempre uma string, e `args` para os argumentos, que é uma tupla de 0 ou mais expressões. Por "expressão", quero dizer ou uma instância de `Expr`, ou um número. Vamos dar uma olhada nos campos para alguns exemplos de `Expr`:

In [5]:
sentence = P & ~Q

sentence.op

'&'

In [6]:
sentence.args

(P, ~Q)

In [7]:
P.op

'P'

In [8]:
P.args

()

In [9]:
Pxy = P(x, y)

Pxy.op

'P'

In [10]:
Pxy.args

(x, y)

É importante notar que a classe `Expr` não define a * lógica * das frases da Lógica Proposicional; Apenas dá-lhe uma maneira de * representar * expressões. Pense em um `Expr` como uma árvore de sintaxe abstrata  (https://en.wikipedia.org/wiki/Abstract_syntax_tree). Cada um dos `args` em um` Expr` pode ser um símbolo, um número ou um `Expr` aninhado. Podemos aninhar estas árvores a qualquer profundidade. Aqui está um deply aninhado `Expr`:

In [11]:
3 * f(x, y) + P(y) / 2 + 1

(((3 * f(x, y)) + (P(y) / 2)) + 1)

## Operadores para Construir Sentenças Lógicas

Aqui está uma tabela dos operadores que podem ser usados para formar frases. Observe que temos um problema: queremos usar operadores do Python para fazer sentenças, de modo que nossos programas (e nossas sessões interativas) mostrem código simples. Mas o Python não permite setas de implicação como operadores, por isso agora precisamos usar uma notação mais detalhada que o Python permite: `| '==>' |` em vez de apenas `==>`. Alternativamente, você sempre pode usar as formas de construtor mais detalhadas de `Expr`:

| Operação                | Símbolo | Python Infix Input | Python Output | Python `Expr` Input
|--------------------------|----------------------|-------------------------|---|---|
| Negação                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P ==> Q` | `Expr('==>', P, Q)`

Aqui está um exemplo de definição de uma frase com uma seta de implicação:

In [12]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

## `expr`: Um atalho para construir frases

Se a notação `| '==>' |` parecer feia para você, você pode usar a função `expr` em vez disso:

In [13]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

`expr` Toma uma string como entrada, e analisa-a em um `Expr`. A string pode conter operadores de seta: `==>`, `<==`, ou `<=>`, que são manipulados como se fossem operadores regulares de infusão de Python. E `expr` define automaticamente quaisquer símbolos, então você não precisa predefinir-los:

In [14]:
expr('sqrt(b ** 2 - 4 * a * c)')

sqrt(((b ** 2) - ((4 * a) * c)))

Por enquanto isso é tudo que você precisa saber sobre `expr`. 

## Bases Proposicionais do Conhecimento: `PropKB`

A classe `Prop KB` pode ser usada para representar uma base de conhecimento de frases proposicionais.

Vemos que a classe `KB` tem quatro métodos, além de` __init__`. Um ponto a ser observado aqui: o método `ask` simplesmente chama o método ` ask_generator`. Assim, este já foi implementado e o que você terá que realmente implementar quando você cria sua própria classe de base de conhecimento (se você quiser, embora eu duvido que você vai precisar, basta usar os que nós criamos para Você), será a função `ask_generator` e não a função` ask` propriamente dita.

A classe `PropKB` agora.
* `__init __ (self, sentence = None)`: O construtor `__init__` cria um único campo ` clauses` que será uma lista de todas as sentenças da base de conhecimento. Note que cada uma dessas frases será uma "cláusula", isto é, uma frase que é composta apenas de literais e "ou".
* `Tell (self, sentence)`: Quando você quiser adicionar uma frase ao KB, use o método `tell`. Esse método usa uma sentença, converte-a em seu CNF, extrai todas as cláusulas e adiciona todas essas cláusulas ao campo `clauses`. Assim, você não precisa se preocupar com `tell`ing apenas cláusulas para a base de conhecimento. Você pode "dizer" à base de conhecimento uma frase de qualquer forma que desejar; Convertê-lo para CNF e adicionando as cláusulas resultantes serão tratadas pelo método `tell`.
* `Ask_generator (self, query)`: A função `ask_generator` é usada pela função ` ask`. Ele chama a função `tt_entails`, que por sua vez retorna ` True` se a base de conhecimento implicar query e `False` caso contrário. O próprio `ask_generator` devolve um dict` {} `vazio se a base de conhecimento implicar a consulta e ` None` caso contrário. Isso pode parecer um pouco estranho para você. Afinal, faz mais sentido apenas retornar `True` ou ` False` em vez de `{}` ou `None` Mas isso é feito para manter a consistência com a maneira como as coisas estão na Lógica de Primeira Ordem, onde, Uma função `ask_generator`, é suposto retornar todas as substituições que tornam a consulta verdadeira. Daí o dict, para retornar todas essas substituições. Eu estarei usando a função `ask` que retorna `True` ou `False`, mas se você não gostar disso, você sempre pode usar a função ` ask_if_true ` que retorna um `True` ou um ` False `
* `Retract (self, sentence)`: Esta função remove todas as cláusulas da sentença dada, a partir da base de conhecimento. Como a função `tell`, você não precisa passar cláusulas para removê-los da base de conhecimento; Qualquer sentença vai fazer bem. A função cuidará de converter essa frase em cláusulas e, em seguida, removê-las.

### Nos próximos vídeos você verá como implementar uma base de conhecimento para criar um módulo de Inteligência Artificial em um Game desenvolvido em Python.

## Fim